finished date: 2023.02.11

first attempt  
initially tried to compute the average using integration, realized that I run into divergence problem

second attempt  
monte carlo integration, worked well enough that I decided this is good enough for now

In [2]:
import scipy as sp
import numpy as np
from scipy import integrate

In [11]:
R = 10.0
num_samples = int(1E4)
r = np.linspace(0.0, R, num_samples)
theta = np.linspace(0.0, sp.pi, num_samples)
phi = np.linspace(0.0, 2*sp.pi, num_samples)

In [12]:
def p(radius, angle_1, angle_2):
    return r**2 * sp.sin(angle_1)

In [1]:
def spher_to_cart(r, theta, phi):
    x = r * sp.sin(theta) * sp.cos(phi)
    y = r * sp.sin(theta) * sp.sin(phi)
    z = r * sp.cos(theta)
    return [x,y,z]

In [2]:
def distance_between_two_spherical_points(r_1, theta_1, phi_1, r_2, theta_2, phi_2):
    x_1, y_1, z_1 = spher_to_cart(r_1, theta_1, phi_1)
    x_2, y_2, z_2 = spher_to_cart(r_2, theta_2, phi_2)
    return sp.sqrt((x_1 - x_2)**2 + (y_1 - y_2)**2 + (z_1 - z_2)**2)

In [3]:
def probability_density_uniform_spher(r_1, theta_1, phi_1, r_2, theta_2, phi_2):
    return r_1**2 * sp.sin(theta_1) * r_2**2 * sp.sin(theta_2)

In [4]:
def potential_weighted(r_1, theta_1, phi_1, r_2, theta_2, phi_2):
    return probability_density_uniform_spher(r_1, theta_1, phi_1, r_2, theta_2, phi_2) * (1/distance_between_two_spherical_points(r_1, theta_1, phi_1, r_2, theta_2, phi_2))

In [6]:
integrate.nquad(potential_weighted, [[0, 1], [0, sp.pi], [0, 2*sp.pi], [0, 1], [0,sp.pi], [0, 2*sp.pi]])

/var/folders/r6/dm5jwjfs0fz2920832swzf8h0000gn/T/ipykernel_51965/1777660724.py:2: DeprecationWarning: scipy.sin is deprecated and will be removed in SciPy 2.0.0, use numpy.sin instead
  r_1**2 * sp.sin(theta_1) * r_2**2 * sp.sin(theta_2)
/var/folders/r6/dm5jwjfs0fz2920832swzf8h0000gn/T/ipykernel_51965/1285322966.py:2: DeprecationWarning: scipy.cos is deprecated and will be removed in SciPy 2.0.0, use numpy.cos instead
  x = r * sp.sin(theta) * sp.cos(phi)
/var/folders/r6/dm5jwjfs0fz2920832swzf8h0000gn/T/ipykernel_51965/1285322966.py:3: DeprecationWarning: scipy.sin is deprecated and will be removed in SciPy 2.0.0, use numpy.sin instead
  y = r * sp.sin(theta) * sp.sin(phi)
/var/folders/r6/dm5jwjfs0fz2920832swzf8h0000gn/T/ipykernel_51965/1285322966.py:4: DeprecationWarning: scipy.cos is deprecated and will be removed in SciPy 2.0.0, use numpy.cos instead
  z = r * sp.cos(theta)
/var/folders/r6/dm5jwjfs0fz2920832swzf8h0000gn/T/ipykernel_51965/2523785570.py:4: DeprecationWarning: scipy.sq

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [3]:
import math
import random
import mcint

In [28]:
"""
the integrand and the sampler for two points uniformly distributed inside a unit sphere
expected value is 6/5 from expected value 6/(5*a) * 1/2 * Z * (Z-1)
Z: num of points
a: radius of the sphere
with Z = 2, a = 1
"""
def integrand_two_points(x):
    x_1 = x[0]
    x_2 = x[1]
    dist = np.linalg.norm(x_1 - x_2)
    return 1/dist

def sampler_two_points():
    while True:
        # two 3d random vector in with each xyz component in [-1, 1)
        x_1 = 2*np.random.rand(3)-1
        x_2 = 2*np.random.rand(3)-1
        x_1_norm = np.linalg.norm(x_1)
        x_2_norm = np.linalg.norm(x_2)
        if x_1_norm < 1 and x_2_norm < 1:
            yield(x_1, x_2)


In [29]:
result_two_points, error_two_points = mcint.integrate(integrand_two_points,sampler_two_points(),n=100000)

In [30]:
result_two_points

1.1996567905111803

In [31]:
error_two_points

0.004637706304601202

In [41]:
"""
the integrand and the sampler for three points uniformly distributed inside a sphere
"""
def integrand_three_points(x):
    # huh... I guess the factor Z*(Z-1) comes directly from the number of distances you have to compute for a given configuration
    x_1 = x[0]
    x_2 = x[1]
    x_3 = x[2]
    dist_1 = np.linalg.norm(x_1 - x_2)
    dist_2 = np.linalg.norm(x_2 - x_3)
    dist_3 = np.linalg.norm(x_1 - x_3)
    return 1/dist_1 + 1/dist_2 + 1/dist_3

def sampler_three_points():
    while True:
        # three 3d random vector in with each xyz component in [-1, 1)
        x_1 = 2*np.random.rand(3)-1
        x_2 = 2*np.random.rand(3)-1
        x_3 = 2*np.random.rand(3)-1
        x_1_norm = np.linalg.norm(x_1)
        x_2_norm = np.linalg.norm(x_2)
        x_3_norm = np.linalg.norm(x_3)
        if x_1_norm < 1 and x_2_norm < 1 and x_3_norm < 1:
            yield(x_1, x_2, x_3)


In [42]:
result_three_points, error_three_points = mcint.integrate(integrand_three_points, sampler_three_points(), n=100000)

In [43]:
result_three_points

3.5979638266947034

In [44]:
error

0.004685685177240609